In [ ]:
import cv2
import mediapipe as mp

# Mediapipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# 카메라 열기
cap = cv2.VideoCapture(0)

# 모자이크 크기
mosaic_size = 100

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 손 감지
    results = hands.process(frame)

    # 가운데 손가락이 감지되면
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 가운데 손가락의 끝점 좌표 얻기
            tip_of_middle = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            tip_x, tip_y = int(tip_of_middle.x * frame.shape[1]), int(tip_of_middle.y * frame.shape[0])

            # 검지
            tip_of_index = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            tip_x1, tip_y1 = int(tip_of_index.x * frame.shape[1]), int(tip_of_index.y * frame.shape[0])

            # 약지
            tip_of_ring = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            tip_x2, tip_y2 = int(tip_of_ring.x * frame.shape[1]), int(tip_of_ring.y * frame.shape[0])

            print(f'가운데 손가락 {tip_x}, {tip_y}')
            print(f'검지 {tip_x1}, {tip_y1}')
            print(f'약지 {tip_x2}, {tip_y2}')
            print()

            print(tip_y2 - tip_y, tip_y1 - tip_y)
            
            if tip_y2 - tip_y >= 170 and tip_y1 - tip_y >= 130:
                # 모자이크 처리
                roi = frame[tip_y - mosaic_size:tip_y + mosaic_size, tip_x - mosaic_size:tip_x + mosaic_size]
                roi = cv2.resize(roi, (mosaic_size * 2, mosaic_size * 2), interpolation=cv2.INTER_NEAREST)
                frame[tip_y - mosaic_size:tip_y + mosaic_size, tip_x - mosaic_size:tip_x + mosaic_size] = roi

        # 손 표시
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # 화면 표시
    cv2.imshow("Hand Tracking", frame)

    # 종료 조건
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [16]:
import cv2
import mediapipe as mp
import numpy as np
import time

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.1)  # 최소 신뢰도 낮춤
mp_drawing = mp.solutions.drawing_utils

# 플레이어 닉네임 입력
player_name = input("게임을 플레이할 닉네임을 입력하세요: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 공 설정
ball_pos = np.array([320, 240])
ball_vel = np.array([0, 0])
ball_radius = 15
gravity = 0.5

# 점수 초기화
score = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    finger_tip = None  # 검지 손가락 끝 좌표 초기화
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for id, lm in enumerate(hand_landmarks.landmark):
                if id == 8:  # 검지 손가락 끝
                    finger_tip = np.array([lm.x, lm.y]) * [frame.shape[1], frame.shape[0]]
                    if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                        ball_vel = (ball_pos - finger_tip) * 0.1
                        ball_vel[1] -= 10  # 손가락에 닿으면 위로 튕겨나감
                        ball_touched = True

    # 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity  # 중력 적용
    ball_pos = ball_pos + ball_vel
    ball_pos = ball_pos.astype(int)  # ball_pos를 정수형으로 변환

    # 화면 범위 내에서 공 유지
    if ball_pos[0] < ball_radius or ball_pos[0] > frame.shape[1] - ball_radius:
        ball_vel[0] *= -1

    if ball_pos[1] < ball_radius:
        ball_vel[1] *= -1

    # 화면 바닥에 떨어졌을 때 게임 종료
    if ball_pos[1] > frame.shape[0] - ball_radius:
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('손가락으로 공 튕기기', frame)
        cv2.waitKey(2000)  # 2초 동안 메시지 표시 후 종료
        break

    # 점수 업데이트
    if ball_touched:
        score += 1

    # 공 그리기
    cv2.circle(frame, tuple(ball_pos.astype(int)), ball_radius, (0, 255, 0), -1)

    # 검지 손가락 끝에 빨간색 원 그리기
    if finger_tip is not None:
        cv2.circle(frame, tuple(finger_tip.astype(int)), 10, (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s point: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('손가락으로 공 튕기기', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
print(f"게임 종료! {player_name}의 최종 점수: {score}")
cap.release()
cv2.destroyAllWindows()


게임 종료! qpqpq의 최종 점수: 6


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)  # 최소 신뢰도 낮춤
mp_drawing = mp.solutions.drawing_utils

# 플레이어 닉네임 입력
player_name = input("게임을 플레이할 닉네임을 입력하세요: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 공 설정
ball_pos = np.array([320, 240])
ball_vel = np.array([0, 0])  # X축 방향 속도를 5로 변경, Y축 방향 속도를 -10으로 변경
ball_radius = 15
gravity = 0.5

# 점수 초기화
score = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    finger_tip = None  # 검지 손가락 끝 좌표 초기화
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # 검지 손가락 끝 좌표 찾기
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]

            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.3  # 속도를 더 높임
                ball_vel[1] -= 10  # 손가락에 닿으면 위로 더 빠르게 튕겨나감
                ball_touched = True
    
    # 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity  # 중력 적용
    ball_pos = ball_pos + ball_vel
    ball_pos = ball_pos.astype(int)  # ball_pos를 정수형으로 변환

    print(ball_pos[0], ball_pos[1])

    # 화면 범위 내에서 공 유지
    if ball_pos[0] < ball_radius or ball_pos[0] > frame.shape[1] - ball_radius:
        ball_vel[0] *= -1

    if ball_pos[1] < ball_radius:
        ball_vel[1] *= -1

    elif ball_pos[1] > frame.shape[0] - ball_radius:
        ball_vel[1] = 0  # 바닥에 닿으면 속도를 0으로 설정
        
    # 화면 바닥에 떨어졌을 때 게임 종료
    if ball_pos[1] > frame.shape[0] - ball_radius:
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('finger ball game', frame)
        cv2.waitKey(2000)  # 2초 동안 메시지 표시 후 종료
        break

    # 점수 업데이트
    if ball_touched:
        score += 1

    # 공 그리기
    cv2.circle(frame, tuple(ball_pos.astype(int)), ball_radius, (0, 255, 0), -1)

    # 검지 손가락 끝에 빨간색 원 그리기
    if finger_tip is not None:
        cv2.circle(frame, tuple(finger_tip.astype(int)), 10, (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s point: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('finger ball game', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 처리
print(f"게임 종료! {player_name}의 최종 점수: {score}")
cap.release()
cv2.destroyAllWindows()


In [108]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# 플레이어 닉네임 입력
player_name = input("게임을 플레이할 닉네임을 입력하세요: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()


# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([320, 0])
ball_vel = np.array([0, 5])
ball_radius = 15
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([0, 0])
second_ball_vel = np.array([0, 0])

# 장애물 설정
obstacle_size = np.array([150, 10])  # 장애물의 크기 (가로, 세로)
obstacle_pos = np.array([random.randint(0, 640 - obstacle_size[0]), 
                         random.randint(0, 480 - obstacle_size[1])])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([320, 120])  # 적당한 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = True
    second_ball_touched = True
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if ball_touched:
                    score += 1
                ball_touched = False

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if second_ball_touched:
                    score += 1
                second_ball_touched = False

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] *= -1
    if ball_pos[1] <= ball_radius:
        ball_vel[1] *= -1

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] *= -1
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] *= -1

    # 장애물과의 충돌 검사 및 점수 업데이트
    obstacle_rect = [obstacle_pos[0], obstacle_pos[1], obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]]
    if (ball_pos[0] + ball_radius > obstacle_rect[0] and ball_pos[0] - ball_radius < obstacle_rect[2] and
        ball_pos[1] + ball_radius > obstacle_rect[1] and ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    if second_ball_active and (
        second_ball_pos[0] + ball_radius > obstacle_rect[0] and second_ball_pos[0] - ball_radius < obstacle_rect[2] and
        second_ball_pos[1] + ball_radius > obstacle_rect[1] and second_ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    # 첫 번째 공 그리기
    cv2.circle(frame, tuple(ball_pos), ball_radius, (255, 255, 255), -1)

    # 두 번째 공 그리기
    if second_ball_active:
        cv2.circle(frame, tuple(second_ball_pos), ball_radius, (255, 255, 255), -1)

    # 장애물 그리기 (사각형)
    cv2.rectangle(frame, (obstacle_pos[0], obstacle_pos[1]), (obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]), (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s point: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('finger ball game', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('finger ball game', frame)
        cv2.waitKey(3000)  # 3초 동안 메시지 표시 후 종료
        break

print(f"게임 종료! {player_name}의 최종 점수: {score}")
# 종료 처리
cap.release()
cv2.destroyAllWindows()

게임 종료! player의 최종 점수: 5


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import csv
import datetime

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# 플레이어 닉네임 입력
player_name = input("게임을 플레이할 닉네임을 입력하세요: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()

# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([320, 0])
ball_vel = np.array([0, 5])
ball_radius = 15
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([0, 0])
second_ball_vel = np.array([0, 0])

# 장애물 설정
obstacle_size = np.array([150, 10])  # 장애물의 크기 (가로, 세로)
obstacle_pos = np.array([random.randint(0, 640 - obstacle_size[0]), 
                         random.randint(0, 480 - obstacle_size[1])])

# CSV 파일 관련 설정
csv_file_path = "game_scores.csv"
csv_header = ["Player Name", "Score"]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([320, 120])  # 적당한 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = True
    second_ball_touched = True
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if ball_touched:
                    score += 1
                ball_touched = False

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if second_ball_touched:
                    score += 1
                second_ball_touched = False

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] *= -1
    if ball_pos[1] <= ball_radius:
        ball_vel[1] *= -1

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] *= -1
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] *= -1

    # 장애물과의 충돌 검사 및 점수 업데이트
    obstacle_rect = [obstacle_pos[0], obstacle_pos[1], obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]]
    if (ball_pos[0] + ball_radius > obstacle_rect[0] and ball_pos[0] - ball_radius < obstacle_rect[2] and
        ball_pos[1] + ball_radius > obstacle_rect[1] and ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    if second_ball_active and (
        second_ball_pos[0] + ball_radius > obstacle_rect[0] and second_ball_pos[0] - ball_radius < obstacle_rect[2] and
        second_ball_pos[1] + ball_radius > obstacle_rect[1] and second_ball_pos[1] - ball_radius < obstacle_rect[3]):
        score -= 1

    # 첫 번째 공 그리기
    cv2.circle(frame, tuple(ball_pos), ball_radius, (255, 255, 255), -1)

    # 두 번째 공 그리기
    if second_ball_active:
        cv2.circle(frame, tuple(second_ball_pos), ball_radius, (255, 255, 255), -1)

    # 장애물 그리기 (사각형)
    cv2.rectangle(frame, (obstacle_pos[0], obstacle_pos[1]), (obstacle_pos[0] + obstacle_size[0], obstacle_pos[1] + obstacle_size[1]), (0, 0, 255), -1)

    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s point: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('finger ball game', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        # 게임 종료 시간 기록
        end_time = time.time()

        # 게임 기록을 CSV 파일에 저장
        with open(csv_file_path, mode='a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([player_name, score, datetime.datetime.now()])

        # 게임 종료 메시지 표시
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('finger ball game', frame)
        cv2.waitKey(3000)  # 3초 동안 메시지 표시 후 종료
        break

print(f"게임 종료! {player_name}의 최종 점수: {score}")

f = open('game_scores.csv','r')
rdr = csv.reader(f)

game_data = []
for line in rdr:
    for i in line:
        print(i)

f.close()

# 종료 처리
cap.release()
cv2.destroyAllWindows()

게임 종료! zxcv의 최종 점수: 3
zxcv
3
2024-01-09 16:09:31.655238


In [6]:
print(str(datetime.datetime.now()))

play_time = str(datetime.datetime.now())


2024-01-09 16:11:53.773644
['2', '0', '2', '4', '-', '0', '1', '-', '0', '9', ' ', '1', '6', ':', '1', '1', ':', '5', '3', '.']


In [19]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# 플레이어 닉네임 입력
player_name = input("Enter a nickname for the game: ")

# 카메라 설정
cap = cv2.VideoCapture(0)

# 점수 및 게임 시작 시간 초기화
score = 0
start_time = time.time()

# 공의 이미지 로드 및 크기 조정
ball_image_path = 'soccerball.png'  # 이미지 경로를 정확히 설정하세요.
ball_image = cv2.imread(ball_image_path, -1)  # -1 플래그로 알파 채널 포함 읽기
if ball_image is None:
    print(f"Failed to load image at {ball_image_path}")
    exit()
ball_image = cv2.resize(ball_image, (30, 30))  # 이미지 크기를 공의 크기로 조정
ball_radius = 15

# 첫 번째 공 위치 및 속도 설정
ball_pos = np.array([320, 0])
ball_vel = np.array([0, 5])
gravity = 0.5

# 두 번째 공 관련 변수
second_ball_active = False
second_ball_pos = np.array([0, 0])
second_ball_vel = np.array([0, 0])

# 첫 번째 장애물 설정
obstacle1_pos = np.array([random.randint(100, 540), random.randint(100, 380)])
obstacle1_radius = 30

# 두 번째 장애물 설정
obstacle2_pos = np.array([random.randint(100, 540), random.randint(100, 380)])
obstacle2_radius = 30

# 이미지의 알파 채널로 마스크 생성
ball_mask = ball_image[:, :, 3]
ball_mask_inv = cv2.bitwise_not(ball_mask)
ball_image = ball_image[:, :, :3]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간
    current_time = time.time()

    # 10초 후 두 번째 공 활성화
    if not second_ball_active and current_time - start_time >= 10:
        second_ball_active = True
        second_ball_pos = np.array([random.randint(ball_radius, frame.shape[1] - ball_radius), 0])  # 화면 내에서 시작 위치 설정
        second_ball_vel = np.array([0, 5])  # 초기 속도 설정

    frame = cv2.flip(frame, 1)  # 좌우 반전
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # 손가락 위치 찾기
    ball_touched = False
    second_ball_touched = False
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 검지 손가락 끝의 랜드마크만 찾기
            index_finger_tip = hand_landmarks.landmark[8]
            finger_tip = np.array([index_finger_tip.x, index_finger_tip.y]) * [frame.shape[1], frame.shape[0]]
            cv2.circle(frame, (int(finger_tip[0]), int(finger_tip[1])), 10, (255, 0, 0), -1)

            # 검지 손가락 끝과 첫 번째 공이 닿았는지 확인
            if np.linalg.norm(finger_tip - ball_pos) < ball_radius + 20:
                ball_vel = (ball_pos - finger_tip) * 0.1
                ball_vel[1] -= 8
                if not ball_touched:
                    score += 1
                ball_touched = True

            # 두 번째 공이 활성화되었을 경우, 검지 손가락과의 충돌 확인
            if second_ball_active and np.linalg.norm(finger_tip - second_ball_pos) < ball_radius + 20:
                second_ball_vel = (second_ball_pos - finger_tip) * 0.1
                second_ball_vel[1] -= 8
                if not second_ball_touched:
                    score += 1
                second_ball_touched = True

    # 첫 번째 공의 물리적 움직임 시뮬레이션
    ball_vel[1] += gravity
    ball_pos = (ball_pos + ball_vel).astype(int)  # 여기에서 타입 변환
    
    # 벽 충돌 검사
    if ball_pos[0] <= ball_radius or ball_pos[0] >= frame.shape[1] - ball_radius:
        ball_vel[0] = -ball_vel[0]
    if ball_pos[1] <= ball_radius:
        ball_vel[1] = -ball_vel[1]
    ball_pos = np.clip(ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)

    # 두 번째 공의 물리적 움직임 시뮬레이션
    if second_ball_active:
        second_ball_vel[1] += gravity
        second_ball_pos = (second_ball_pos + second_ball_vel).astype(int)  # 여기에서 타입 변환
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)
        # 벽 충돌 검사
        if second_ball_pos[0] <= ball_radius or second_ball_pos[0] >= frame.shape[1] - ball_radius:
            second_ball_vel[0] = -second_ball_vel[0]
        if second_ball_pos[1] <= ball_radius:
            second_ball_vel[1] = -second_ball_vel[1]
        second_ball_pos = np.clip(second_ball_pos, ball_radius, np.array([frame.shape[1], frame.shape[0]]) - ball_radius)


    # 장애물 그리기 (동그라미)
    cv2.circle(frame, tuple(obstacle1_pos), obstacle1_radius, (0, 0, 255), -1)
    cv2.circle(frame, tuple(obstacle2_pos), obstacle2_radius, (0, 0, 255), -1)

    # 장애물과의 충돌 검사 및 점수 업데이트 (첫 번째 장애물)
    if np.linalg.norm(ball_pos - obstacle1_pos) < ball_radius + obstacle1_radius:
        score -= 1
        ball_vel = -ball_vel  # 장애물에 닿으면 반대 방향으로 튕겨나감

    if second_ball_active and np.linalg.norm(second_ball_pos - obstacle1_pos) < ball_radius + obstacle1_radius:
        score -= 1
        second_ball_vel = -second_ball_vel  # 장애물에 닿으면 반대 방향으로 튕겨나감

    # 장애물과의 충돌 검사 및 점수 업데이트 (두 번째 장애물)
    if np.linalg.norm(ball_pos - obstacle2_pos) < ball_radius + obstacle2_radius:
        score -= 1
        ball_vel = -ball_vel

    if second_ball_active and np.linalg.norm(second_ball_pos - obstacle2_pos) < ball_radius + obstacle2_radius:
        score -= 1
        second_ball_vel = -second_ball_vel

    # 첫 번째 공 그리기
    if 0 <= ball_pos[1] - ball_radius and ball_pos[1] + ball_radius < frame.shape[0] and 0 <= ball_pos[0] - ball_radius and ball_pos[0] + ball_radius < frame.shape[1]:
        ball_center = (ball_pos[0] - ball_radius, ball_pos[1] - ball_radius)
        ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
        ball_alpha_s = ball_mask / 255.0
        ball_alpha_l = 1.0 - ball_alpha_s

        for c in range(0, 3):
            ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                ball_alpha_l * ball_rect[:, :, c])

    # 두 번째 공 그리기
        if 0 <= second_ball_pos[1] - ball_radius and second_ball_pos[1] + ball_radius < frame.shape[0] and 0 <= second_ball_pos[0] - ball_radius and second_ball_pos[0] + ball_radius < frame.shape[1]:
            ball_center = (second_ball_pos[0] - ball_radius, second_ball_pos[1] - ball_radius)
            ball_rect = frame[ball_center[1]:ball_center[1]+2*ball_radius, ball_center[0]:ball_center[0]+2*ball_radius]
            ball_alpha_s = ball_mask / 255.0
            ball_alpha_l = 1.0 - ball_alpha_s

            for c in range(0, 3):
                ball_rect[:, :, c] = (ball_alpha_s * ball_image[:, :, c] +
                                    ball_alpha_l * ball_rect[:, :, c])


    # 점수 표시
    cv2.putText(frame, f'{player_name}\'s score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # 화면에 결과 표시
    cv2.imshow('Finger Ball Game', frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

 # 바닥에 닿으면 게임 종료
    if ball_pos[1] >= frame.shape[0] - ball_radius or (second_ball_active and second_ball_pos[1] >= frame.shape[0] - ball_radius):
        print(f"Game Over! {player_name}'s final score: {score}")
        cv2.putText(frame, 'Game Over', (frame.shape[1] // 2 - 100, frame.shape[0] // 2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow('Finger Ball Game', frame)
        cv2.waitKey(2000)  # Wait for 2 seconds
        break

# 종료 처리
cap.release()
cv2.destroyAllWindows()

Game Over! 's final score: 0
